In [1]:
import numpy as np
import tensorflow  as tf
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import librosa
from google.colab import files
from google.colab import drive
import glob
import pickle
from statistics import mean
import keras
from keras.layers.core import Dense, Permute, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.regularizers import l2
from sklearn.metrics import confusion_matrix


np.random.seed(42)

Using TensorFlow backend.


In [2]:
drive.mount('/content/Drive/',force_remount =  True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/Drive/


In [0]:
dirpath = "/content/Drive/My Drive/DL project/New Files/Mel Data/" ####Change

In [0]:
# !pip install librosa.display.specshow

In [0]:
#Loading training pickle files
sound_lists_pkl_in = open(dirpath+"Catalan_mel_2500.pkl", "rb")
ca_data_mel = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"French_mel_2500.pkl", "rb")
fr_data_mel = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"English_mel_2500.pkl", "rb")
en_data_mel = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"German_mel_2500.pkl", "rb")
de_data_mel = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"Italian_mel_2500.pkl", "rb")
it_data_mel = pickle.load(sound_lists_pkl_in)



In [0]:
#Appending italian with zeros
for i in range(len(it_data_mel['mel'])):
  max_len = 242
  zeros = np.zeros((max_len-it_data_mel['mel'][i].shape[0],it_data_mel['mel'][i].shape[1]))
  it_data_mel['mel'][i] = np.concatenate((it_data_mel['mel'][i],zeros), axis=0)

In [0]:
def no_of_row(dict):
  rows = []
  for i in range(len(dict['mel'])):
    r = dict['mel'][i].shape[0]
    rows.append(r)
  print(min(rows),max(rows))
  

In [8]:
no_of_row(it_data_mel)
no_of_row(de_data_mel)
no_of_row(en_data_mel)
no_of_row(fr_data_mel)
no_of_row(ca_data_mel)

242 242
242 242
242 242
242 242
242 242


In [0]:
def data_creation(dict):
  data=[]
  mel=[]
  for i in range(2500):
    s = dict['mel'][i]
    mel.append(s)
  mel = np.array(mel)
  for i in range(len(mel)):
    ele = np.array([mel[i],dict['labels'][i]])
    data.append(ele)
  tr_data = np.array(data[:2000])
  te_data = np.array(data[2000:])
  return tr_data, te_data
    
                   
  

In [0]:
ca_data_mel_tr, ca_data_mel_te = data_creation(ca_data_mel)

In [11]:
len(ca_data_mel_te)

500

In [0]:
#Train Data
# ca_data_mel_tr, ca_data_mel_te = data_creation(ca_data_mel)
en_data_mel_tr, en_data_mel_te = data_creation(en_data_mel)
fr_data_mel_tr, fr_data_mel_te = data_creation(fr_data_mel)
it_data_mel_tr, it_data_mel_te = data_creation(it_data_mel)
de_data_mel_tr, de_data_mel_te = data_creation(de_data_mel)

In [13]:
de_data_mel_tr[0]

array([array([[7.80457434e-07, 8.16006239e-08, 4.96844250e-07, ...,
        8.02388426e-09, 3.53154418e-09, 2.50996637e-10],
       [5.23175602e-07, 2.31100796e-07, 5.46005448e-07, ...,
        6.57729968e-09, 3.46898831e-09, 2.85545132e-10],
       [1.04983865e-06, 2.49915937e-06, 1.65298861e-06, ...,
        5.63895780e-09, 2.19717601e-09, 1.69920772e-10],
       ...,
       [2.71399047e-04, 1.93461394e-04, 2.71555819e-04, ...,
        2.36638217e-03, 3.74422654e-04, 1.84601619e-05],
       [3.68046983e-04, 7.37894870e-04, 8.32881879e-04, ...,
        2.21180844e-03, 3.04823619e-04, 1.94661517e-05],
       [7.51647510e-04, 1.16108362e-03, 1.37731047e-03, ...,
        9.82638044e-04, 1.89948630e-04, 2.71922827e-05]]),
       'German'], dtype=object)

In [0]:
final_data_tr = np.concatenate((ca_data_mel_tr, fr_data_mel_tr, en_data_mel_tr, de_data_mel_tr, it_data_mel_tr), axis = 0)

In [0]:
final_data_te = np.concatenate((ca_data_mel_te, fr_data_mel_te, en_data_mel_te, de_data_mel_te, it_data_mel_te), axis = 0)

In [16]:
#QC
print(np.unique(final_data_tr[:,1], return_counts = True))
print(np.unique(final_data_te[:,1], return_counts = True))


(array(['Catalan', 'English', 'French', 'German', 'Italian'], dtype=object), array([2000, 2000, 2000, 2000, 2000]))
(array(['Catalan', 'English', 'French', 'German', 'Italian'], dtype=object), array([500, 500, 500, 500, 500]))


In [17]:
#QC
print(final_data_tr.shape)
print(final_data_te.shape)

(10000, 2)
(2500, 2)


In [0]:
del(ca_data_mel_te)
del(it_data_mel_te)
del(fr_data_mel_te)
del(en_data_mel_te)
del(de_data_mel_te)

In [0]:
del(ca_data_mel_tr)
del(it_data_mel_tr)
del(fr_data_mel_tr)
del(en_data_mel_tr)
del(de_data_mel_tr)

In [0]:
np.random.shuffle(final_data_tr)
np.random.shuffle(final_data_te)

In [21]:
#Labels

labels_tr =[]
for i in range(len(final_data_tr)):

      labels_tr.append(final_data_tr[i][1])  

labels_tr = np.array(labels_tr)
labels_tr

classnames, indices = np.unique(labels_tr, return_inverse=True)
print(indices)
print(classnames)

final_labels_tr = indices

[3 1 0 ... 1 0 3]
['Catalan' 'English' 'French' 'German' 'Italian']


In [22]:
labels_te =[]
for i in range(len(final_data_te)):

      labels_te.append(final_data_te[i][1])  

labels_te = np.array(labels_te)
labels_te

classnames, indices = np.unique(labels_te, return_inverse=True)
print(indices)
print(classnames)

final_labels_te = indices

[3 1 2 ... 2 3 3]
['Catalan' 'English' 'French' 'German' 'Italian']


In [23]:
final_X_tr =[]
for i in range(len(final_data_tr)):

      final_X_tr.append(final_data_tr[i][0])  

final_X_tr = np.array(final_X_tr)
final_X_tr[0].shape


(242, 128)

In [24]:
final_X_te =[]
for i in range(len(final_data_te)):

      final_X_te.append(final_data_te[i][0])  

final_X_te = np.array(final_X_te)
final_X_te[0].shape


(242, 128)

### Keras - CRNN

In [0]:
from keras.layers.core import Dense, Permute, Reshape, Flatten
from keras.layers import Dense, Permute, Reshape, Input
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, TimeDistributed
from keras import layers
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.layers import Input
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model, load_model

In [0]:
# input_shape = (40,972,1)
input_shape2 = (247,129,1)


In [0]:
del(final_data_tr)
del(final_data_te)

In [0]:
tx = np.array(final_X_tr[:8000])
cvx = np.array(final_X_tr[8000:])
ty = np.array(final_labels_tr[:8000])
cvy = np.array(final_labels_tr[8000:])

In [0]:
del(final_labels_tr)
del(final_X_tr)

In [0]:
cvy_cat = to_categorical(cvy)
ty_cat = to_categorical(ty)

In [0]:
def corr_shape(arr):
  new_arr = []
  for i in range(len(arr)):
    new_arr.append(arr[i])
  new_arr = np.array(new_arr)
  print(new_arr.shape)
  return new_arr

In [32]:
tx = corr_shape(tx)
cvx = corr_shape(cvx)

(8000, 242, 128)
(2000, 242, 128)


In [0]:
# input_shape = (40,972,1)
input_shape2 = (242,128,1)

In [45]:
#Inception model
input_tensor = Input(shape=input_shape2)
inception_model = InceptionV3(include_top=False, weights=None, input_tensor=input_tensor)

# for layer in inception_model.layers:
#   layer.trainable = False
  
inception = inception_model.output
x = inception
#x = GlobalAveragePooling2D()(x)
x = Dropout(0.9)(x)

# model.add(Permute((2,1,3)))

# bs, x, y, c = model.layers[-1].output_shape

# model.add(Reshape([x,y*c]))

# model.add(LSTM(512, activation="relu",dropout = 0.7, return_sequences=False))

# x = Permute((2, 1, 3))(x)

_x, _y, _c = [int(s) for s in x._shape[1:]]
x = Reshape((_x, _y*_c))(x)

x = LSTM(256, return_sequences=True)(x)

x = LSTM(512, activation="tanh", return_sequences=False, dropout=0.7)(x)

# x = Bidirectional(LSTM(512, return_sequences=False), merge_mode="concat")(x)

# x = Flatten()(x)

x = Dense(1024, activation="relu")(x)

predictions = Dense(5, activation="softmax")(x)

model = Model(inputs = inception_model.input, output = predictions)




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.adam(lr=0.0002),
             metrics=["accuracy"])

In [0]:
model.fit(tx.reshape(-1,242,128,1), ty_cat,
          batch_size=100,
          epochs=100,
          verbose=1,
          validation_data=(cvx.reshape(-1,242,128,1),cvy_cat)
         )
          

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 77s 10ms/step - loss: 1.5853 - acc: 0.2466 - val_loss: 1.6910 - val_acc: 0.2525
Epoch 2/100
8000/8000 [==============================] - 45s 6ms/step - loss: 1.4918 - acc: 0.3143 - val_loss: 1.6975 - val_acc: 0.2505
Epoch 3/100
8000/8000 [==============================] - 45s 6ms/step - loss: 1.4281 - acc: 0.3529 - val_loss: 2.5273 - val_acc: 0.2100
Epoch 4/100
8000/8000 [==============================] - 45s 6ms/step - loss: 1.3728 - acc: 0.4022 - val_loss: 1.8149 - val_acc: 0.3495
Epoch 5/100
8000/8000 [==============================] - 45s 6ms/step - loss: 1.3069 - acc: 0.4385 - val_loss: 2.7768 - val_acc: 0.2220
Epoch 6/100
8000/8000 [==============================] - 45s 6ms/step - loss: 1.2557 - acc: 0.4772 - val_loss: 2.1664 - val_acc: 0.3110
Epoch 7/100
8000/8000 [==============================] - 45s 6ms/step - loss: 1.2379 - acc: 0.4941 - val_loss: 2.0915 - val_acc: 0.3365

In [0]:
 # model = load_model('Drive/My Drive/DL project/my_model.h5') ####

In [0]:
y_pred = model.predict(cvx.reshape(-1,242,128,1))
matrix = confusion_matrix(cvy_cat.argmax(axis=1), y_pred.argmax(axis=1))

In [40]:
matrix

array([[178,  96,  16,  77,  33],
       [ 15, 244,   2, 115,  21],
       [ 28,  96,  85, 151,  23],
       [ 19,  73,   7, 301,  19],
       [ 11,  69,  24,  57, 240]])